### https://leetcode.com/problems/network-delay-time/

In [ ]:
class Solution:
    def networkDelayTime(self, times: List[List[int]], n: int, k: int) -> int:
        edges = collections.defaultdict(list)
        for u,v,w in times:
            edges[u].append((v,w))
        minHeap = [(0,k)]
        visit = set()
        t = 0
        while minHeap:
            w1,n1 = heapq.heappop(minHeap)
            if n1 not in visit:
                visit.add(n1)
                t = max(t,w1)
                
                for n2, w2 in edges[n1]:
                    if n2 not in visit:
                        heapq.heappush(minHeap,(w1 + w2,n2))
        
        return t if len(visit) == n else -1

# Algorithm

#### Dijkstra's Algorithm: 
We can use use Dijkstra's algorithm to solve single source shortest path problems for weighted directed graphs with non-negative weight edges. We will implement a greedy algorithm whereby we will choose from the min of the path costs to particular target vertices so that when the adjacent nodes to that particular target vertex are considered (i.e nodes where there is a directed edge incident to it from the vertex with the min path cost that we have encountered from all path cost options), we can add the min path cost to the edge weight along the directed edge reaching the adjacent node to compute potentially another minimum path cost (i.e sum of edge weights along certain path) so the adjacent node. By repeating this process for each process for each node, at the end, we will end up with the min time for a signal to start from the source node and reach each of the target vertices. 

We will need to maintain an adjacency list and min heap so that whenever we remove the top element from the min heap, we can consider the node and use it as a 1-based index into the adjacency list (there will be an offset of 1 since the adjacency list is 0-indexed), and retrieve its adjacent nodes (the nodes to which there is a directed edge incident to originating from the node that was used as index into the adjacency list). We will also need to maintain a list of min signal times, which intuitively will retain the minimum time we have encountered for a signal to reach the destination node starting from the source node considering all of the paths we have seen thus far. At any particular moment of time, this list may not be holding onto the ultimate minimum time to reach the destination node. We can only be sure this time is the utlimate minimum at the very end (i.e. after there are no more elements left in the min heap). The idea is that our min heap will store a tuple pair where the first entry of the pair is the path cost to reach a particular node and the label of the node is the second entry. Note that we must be careful to not confuse this path cost, when deleted from the min heap, as the minimum path cost we have seen thus far for all paths from the source node to destination vertex. When we remove the top element from the min heap, we iwll consider all adjacent nodes with directed edges originating from the node we had jsut popped, so we may discover that the path the time for the signal to reach that node along a particular path (i.e the first entry of the tuple pair) plus the weight along the directed edge may not yield a new minimum path cost for the adjacent node, so we will have to update our list of min signal times and insert the adjacent node into the min heap. However, a tuple pair mau already exist in the min heap whereby the second entry is the adjacency node we have just pushed into the min heap. We have no way of finding it unless we keep removing the top element of the heap, and in the process we may remove some relevant path costs. So, we will have to leave it in there, and just be careful with our wording when describing what the top element is that is removed from the min heap. It is simply just a path cost (sum of edge weights along a particular path). This path cost may be the minimum of all paths we have seen thus far from the source vertex to the destination vertex. It may not be either. The only way we can be sure is comparing it with the list entry from the min signal cost and check if they are equal. We can use the second entry of the tuple pair as a 1-based index into this list. If they are not equal, we shouldn't do anything (just proceed to remove the next top element from the min heap if it is not empty). Otherwise, we know that the path cost we see is the minimum time to for a signal to go from the source vertex to the destination vertex along all paths we have seen thus far, so we can be greedy and interpret it as the ultimate minimum time, so all times to reach the adjacent nodes going through the currnet target vertex will factor in this 'ultimate minimum time', so if we were to find a lower minimum time for the adjacent node compared to what the min signal time list is currently storing, we can update it as the new minimum time to reach that adjacency node.

#### Algorithm Explanation
1. Heapify a list solely containing a tuple where the first entry is 0 and the second entry is the source vertex into a min heap. Declare and initialize a list at size V for the min signal time where V is the number of vertices in the graph, or V = n. Initialize each of the list entries to the max int since we have not yet found a path from the source vertex to the destination node. Lastly, create an adjacency list where each of the vertices is a 1-based index into the first dimension of the 2d matrix. The adjacency list will each of the nodes in another vertex to which there is a directed edge pointing to it from the vertex that is used as an index into the first dimension. Update the list of min signal times so that the entry corresponding to the source node is equal to 0. The source node that emits the signal will already have the signal at the very start, and there are no other vertices in the path, so the min path cost to reach the source vertex from the source vertex is 0. 
2. As long as the min heap isn't empty, remove the top element from the min heap. Compare the first entry of the tuple pair to the list entry of min signal times corresponding to the second entry of the tuple pair (which will be used as a 1-based index into this list). If they are equal, retrieve a list of adjacent nodes by indexing into the adjacency list with the current node we popped from the min heap as a 1-based index. Go through each of these adjacency nodes and add the min path cost to reach the target vertex that we had just deleted from the min heap to the weight along the directed edge incident to the particular adjacent node. If it is less than the list entry of min signal times corresponding to the adjacent node, update the list entry to reflect this new minimum path cost and then insert a new tuple pair into the min heap where the first entry is the minimum past cost we just completed for the path we had seen thus far from the source to adjacenct node and the second entry is the adjacent node itself.
3. Once the min heap is empty, go through the list of min signal-times and check if any of the entries are still to the MAX_INT. If so, there is no path between the source vertex and that particular destination vertex, so there can't be a min time for all n nodes to receive the signal if even one of the n nodes isn't receiving the signal. So returning -1 in this case. Otherwise, return the max entry of the list since the min heap since the min times for all of the n nodes to receive the signal must be considered, so the node taking the longest time for the minimum path cost along all paths from the source vertex to that particular destination will dominate the total minimum time. 


Runtime: We first loop through the input list of times. The 2d matrix consists of each of the directed edges. We must have the following information for each edge: the source vertex, destination vertex, and the time taken for the signal to go from the source to destination vertex (i.e edge weight). The input 2d matrix of times has all this information, and each of its entries (list themselves) represent each of the edges. Therefore, the first loop will total E iterations where E is the number of edges in the graph and is equal to length of the times. We heapify a list containing a single tuple entry whose elements consist of the source vertex and the minimum path cost from the source vertex to itself, which is just 1. For a list of V entries, the time complexity for heapification would be linear, so since we are guaranteed to heapify a list containing only a single tuple pair, the time complexity for heapification can be considered constant. Next, we have an outer while loop and inner for loop. The while loop will continue so long as the min heap isn't empty. We know, however, the total number of for loop iterations across all outer while loop iterations, so we can consider its runtime independently, and this runtime will be the total runtime for this portion of the algorithm. We go through each of the adjacent nodes for each of the destination vertices (including the source vertex). As long as the destination vertex is connected to the source vertex (meaning there's a path from the estimation to source vertex), we're guaranteed to traverse the outgoing edges from each of the destination vertices at least once. In the worst-case scenario, each of the path costs we insert into the min heap are indeed the smallest path costs, so we have to go through each of the adjacent nodes for all edges. The total number of inner loop iterations will sum to E, so this would also be the runtime for the outer while loop where we continue removing the top elements from the min heap so long as it isn't empty. So, the max heap of elements that will be inserted into our min heap is E. Each insertion and deletion operation costs us O(logE) time since for an iteration operation, we would need to satisfy the second property of min heaps that the value of each node should be no greater than the values of its children nodes. So, starting from the newly inserted node (the first property of min heaps are that they are complete binary trees), we would need to check the parent of it and the parent of the parent node to ensure that they satisfy this second condition. So, we will have to traverse a path of length h, which we know for a complete binary tree to be log(E) for a complete binary tree of E vertices. Likewise, for the deletion operation, after swapping the root node and the very last index node (that we would encounter in a BFS traversal), and deleting the root node, we still need to once again traverse the path of length h to verify the second property of a complete binary tree, which would require O(logE) time. We perform a deletion operation for the top element in the min heap until it is empty (every element) and may couple it with an insertion operation in the inner for loop, which we know to have a runtime of O(E), so the overall runtime is O(V + ElogE). However, notice that that an upper bound for E is 2*V. So we can substitute this into the expression. It simplifies down to O(V + ElogV^2). So the overall runtime becomes O(ElogV). 

Space Complexity: The adjacency list will require O(E) space since for every vertex in the graph, we will store each vertex to which there is a directed edge incident to it originating from the vertex, which is ued as an index into the first dimension of the adjacency list. So, the number of adjacent nodes to all the vertices will be linear to the total number of directed edges in the graph. We also know from above that this max number of elements our min heap will need to store is E, which is the memory footprint for the min heap. Lastly, for our list of min signal items, we will need to retain the min signal time for each vertex, which is used as an index into the first dimension of the list, so we require O(V) space for this list. Our overall memory usage is O(V + 2*E), which can be simplified down to O(V + E). 

### https://www.youtube.com/watch?v=EaphyqKU4PQ